In [2]:
import graphlab
import numpy as np
import matplotlib.pyplot as plt

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to sripriya.kuram@gmail.com and will expire on September 13, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\sripr\AppData\Local\Temp\graphlab_server_1479421993.log.0


In [4]:
sales['floors'] = sales['floors'].astype(float)

In [16]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1
    features = ['constant'] + features
    data = data_sframe[features]
    feature_matrix = data.to_numpy()
    output_array = data_sframe[output].to_numpy()
    return (feature_matrix, output_array)

In [17]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix , weights)
    return predictions

In [18]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


In [19]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


In [20]:
def normalize_features(features):
    norms = np.linalg.norm(features , axis =0)
    normalized_features = features / norms
    return (normalized_features, norms)

In [21]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
print norms

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


In [22]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [23]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [24]:
weights = np.array([1., 4., 1.])

In [38]:
prediction = predict_output(simple_feature_matrix , weights)

In [65]:
ro=[]
for i in range(len(weights)):
        ro_i = np.dot( simple_feature_matrix[:,i] , (output - prediction + weights[i] *simple_feature_matrix[:,i]))
        ro.append(ro_i)
print ro

[79400300.034929186, 87939470.772991002, 80966698.675965697]


In [63]:
80966698.67596525 * 2

161933397.3519305

In [64]:
87939470.772990793 * 2

175878941.5459816

In [68]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix , weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.dot( feature_matrix[:,i] , (output - prediction + weights[i] *feature_matrix[:,i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [69]:
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


In [122]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = np.array(initial_weights)
    converged = False
    while not converged:
        converged = True
        for i in range(len(weights)):
            old_weight = weights[i]
            weights[i] = lasso_coordinate_descent_step(i,feature_matrix,output,weights,l1_penalty)
            #print weights
            change = abs(old_weight - weights[i])
            if change >= tolerance:
                converged = False
        
    return  weights    

In [123]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [124]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [125]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [126]:
weights

array([ 21624998.36636294,  63157246.78545421,         0.        ])

In [127]:

def get_residual_sum_of_squares(predictions, output):
     
    # Then compute the residuals/errors
    residual = output - predictions

    # Then square and add them up
    residual_squared = residual * residual
    
    RSS = residual_squared.sum()

    return(RSS)

In [128]:
predictions = predict_output(normalized_simple_feature_matrix, weights)
RSS = get_residual_sum_of_squares(predictions, output)
print("RSS: $%.6f" % (RSS))

RSS: $1630492481484487.750000


In [129]:
train_data,test_data = sales.random_split(.8,seed=0)

In [130]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [131]:
more_f , o = get_numpy_data(train_data, all_features,'price')
a,b =normalize_features(more_f)

In [132]:
initial_weights = np.zeros(14)
weights1e7 = lasso_cyclical_coordinate_descent(a, o,
                                            initial_weights, 1e7, 1)

In [133]:
weights1e7  # 1478

array([ 24429600.60933308,         0.        ,         0.        ,
        48389174.35227981,         0.        ,         0.        ,
         3317511.16271982,   7329961.9848964 ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

In [134]:
weights1e8 = lasso_cyclical_coordinate_descent(a, o,
                                            initial_weights, 1e8, 1)

In [135]:
weights1e8 #1478

array([ 71114625.75280941,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

In [152]:
weights1e4 = lasso_cyclical_coordinate_descent(a, o,
                                            initial_weights, 1e4, 5e5)

In [153]:
weights1e4

array([ 78564738.47260284, -22097399.12360626,  12791071.78258323,
        93808087.79432736,  -2013172.81209793,  -4219184.78179709,
         6482842.76876443,   7127408.69205861,   5001665.52755265,
        14327517.39644263, -15770958.97405414,  -5159591.03562764,
       -84495341.35537505,   2824439.40779558])

In [146]:
normalized_weights1e4 = weights1e4 / b 
normalized_weights1e7 = weights1e7 / b 
normalized_weights1e8 = weights1e8 / b

In [149]:
print normalized_weights1e7[3]

312.732802348


In [154]:

(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

In [160]:
predictions = predict_output(test_feature_matrix, normalized_weights1e4)
RSS = get_residual_sum_of_squares(predictions, test_output)
print("RSS on TEST data with normalized_weights1e4: $%.6f" % (RSS))

RSS on TEST data with normalized_weights1e4: $228459962944943.218750


In [159]:

predictions = predict_output(test_feature_matrix, normalized_weights1e7)
RSS = get_residual_sum_of_squares(predictions, test_output)
print("RSS on TEST data with normalized_weights1e7: $%.6f" % (RSS))

RSS on TEST data with normalized_weights1e7: $228459962944943.218750


In [158]:

predictions = predict_output(test_feature_matrix, normalized_weights1e8)
RSS = get_residual_sum_of_squares(predictions, test_output)
print("RSS on TEST data with normalized_weights1e8: $%.6f" % (RSS))

RSS on TEST data with normalized_weights1e8: $537166150034084.875000
